In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import json

In [ ]:
# ========================================
# 1. LOAD TEXAS COUNTY BOUNDARIES
# ========================================

url = "https://www2.census.gov/geo/tiger/TIGER2025/COUNTY/tl_2025_us_county.zip"
counties = gpd.read_file(url)

# Filter to Texas only (FIPS code 48)
texas_counties = counties[counties['STATEFP'] == '48'].copy()

print(f"Loaded {len(texas_counties)} Texas counties")
print(texas_counties.head())

In [ ]:
# Data in csv from https://earlyvoting.texas-election.com/Elections/getElectionEVDates.do for 2024 November General election, Official Election Day turnout by 
# land area from https://www.texascounties.net/statistics/landarea.htm
# population https://www2.census.gov/programs-surveys/popest/tables/2020-2024/counties/totals/co-est2024-pop-48.xlsx

# ========================================
# 2. PREPARE DATA
# ========================================
voter_data = pd.read_csv('data/texas_voter_density.csv')

voter_data.columns = [
    'county_name',
    'registered_voters',
    'number_in_person_on_20241105',
    'cumulative_in_person_voters',
    'cumulative_percent_in_person',
    'cumulative_by_mail',
    'cumulative_in_person_and_mail',
    'voter_turnout',
    'land_area_sq_miles',
    'population_density',
    'total_area_sq_miles',
    'total_population'
]

numeric_columns = [
    'registered_voters',
    'number_in_person_on_20241105',
    'cumulative_in_person_voters',
    'cumulative_by_mail',
    'cumulative_in_person_and_mail',
    'land_area_sq_miles',
    'total_area_sq_miles',
    'total_population',
]
# Remove commas and convert to integers
for col in numeric_columns:
    voter_data[col] = voter_data[col].str.replace(',', '').astype(float)

# Handle percentage columns (remove % sign if present)
if voter_data['cumulative_percent_in_person'].dtype == 'object':
    voter_data['cumulative_percent_in_person'] = (
        voter_data['cumulative_percent_in_person']
        .str.replace('%', '')
        .astype(float)
    )
if voter_data['voter_turnout'].dtype == 'object':
    voter_data['voter_turnout'] = (
        voter_data['voter_turnout']
        .str.replace('%', '')
        .astype(float)
    )


voter_data['voter_density'] = voter_data['cumulative_in_person_and_mail']/voter_data['land_area_sq_miles']
voter_data['voter_percent'] = 100*voter_data['cumulative_in_person_and_mail']/voter_data['total_population']
voter_data['voter_percent_clipped'] = ((voter_data['cumulative_in_person_and_mail']/voter_data['total_population']).clip(0,1))*100
voter_data['log_cumulative_in_person_and_mail'] = np.log10(voter_data['cumulative_in_person_and_mail'].replace(0, np.nan))

# Merge with geographic data
texas_counties = texas_counties.merge(
    voter_data, 
    left_on='NAME', 
    right_on='county_name', 
    how='left'
)

In [ ]:
# ========================================
# 3. CREATE INTERACTIVE MAP (with hover)
# ========================================
texas_counties_json = json.loads(texas_counties.to_json())
figA = px.choropleth(
    texas_counties,
    geojson=texas_counties_json,
    locations='GEOID',
    featureidkey='properties.GEOID',
    color='log_cumulative_in_person_and_mail',
    hover_name='county_name',
    hover_data={
        'total_population': ':,.0f',
        'registered_voters': ':,.0f',
        'cumulative_in_person_and_mail': ':,.0f',
        'voter_percent': ':.2f', 
        'land_area_sq_miles': ':,.0f',
        'voter_density': ':.2f',
        'log_cumulative_in_person_and_mail': ':.3f',
        'GEOID': False},
    title='Texas Voter Information by County',
    labels={
        'total_population': 'Population',
        'registered_voters': 'Registered Voters',
        'cumulative_in_person_and_mail': 'Voters',
        'voter_percent': 'Voter Percentage',
        'land_area_sq_miles': 'Land area (mi^2)',    
        'voter_density': 'Voter Density', 
        'log_cumulative_in_person_and_mail': 'Log10(Total Voters)'

    }
)

figB = px.choropleth(
    texas_counties,
    geojson=texas_counties_json,
    locations='GEOID',
    featureidkey='properties.GEOID',
    color='voter_percent_clipped',
    hover_name='county_name',
    hover_data={
        'total_population': ':,.0f',
        'registered_voters': ':,.0f',
        'cumulative_in_person_and_mail': ':,.0f',
        'voter_percent': ':.2f', 
        'voter_percent_clipped': False, 
        'land_area_sq_miles': ':,.0f',
        'voter_density': ':.2f',
        'GEOID': False},
    title='Texas Voter Information by County',
    labels={
        'total_population': 'Population',
        'registered_voters': 'Registered Voters',
        'cumulative_in_person_and_mail': 'Voters',
        'voter_percent': 'Voter %',
        'land_area_sq_miles': 'Land area (mi^2)',   
        'voter_density': 'Voter Density', 
    }
)

# --- Combine both into a single Figure and hide the second by default ---
fig = go.Figure(data=[figA.data[0], figB.data[0]])
fig.data[1].visible = False  # start with first map shown

common_cb = dict(
    len=0.60,
    thickness=20
)

fig.update_geos(
    fitbounds="locations",
    visible=False,
    center=dict(lat=31.5, lon=-99.5),  # Center of Texas
    projection_scale=5  # Zoom in on Texas
)

fig.update_layout(
    height=800, width=1200,
    margin={"r":0,"t":50,"l":0,"b":0},
    title={
        'text': "Texas Voter Analysis Dashboard",
        'x': 0.2,  # Center the title
        'y': 0.99,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 24}  # Optional: adjust font size
    },
    coloraxis_colorbar=dict(x=0.96, y=0.5, yanchor='middle', len=0.6),
    coloraxis1=dict(
        colorscale='YlOrRd',
        colorbar=dict(title='Log10(Total Voters)', x=0.85, **common_cb)
    ),
    coloraxis2=dict(
        colorscale='Blues',
        colorbar=dict(title='Voter Percentage', x=0.85, **common_cb)
    ),

    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=0.5, xanchor="center",
        y=1, yanchor="top",
        showactive=True,
        buttons=[
            dict(
                label="Total Voters",
                method="update",
                args=[
                    {"visible": [True, False]},  # show A, hide B
                    {"title.text": "Texas Voter Analysis Dashboard"}
                ]
            ),
            dict(
                label="Voter Percentage",
                method="update",
                args=[
                    {"visible": [False, True]},  # hide A, show B
                    {"title.text": "Texas Voter Analysis Dashboard"}
                ]
            ),
        ]
    )],


)

fig.data[0].update(coloraxis='coloraxis1')
fig.data[1].update(coloraxis='coloraxis2')



# Save as HTML (interactive with hover)
fig.write_html('outputs/texas_voter_density_interactive.html',include_plotlyjs='cdn')


#for debug
# fig.show()

In [ ]:
texas_counties.columns